In [ ]:
var scenario = "IRU";
var year = 2020;
var month = 12;
var reportingNode = "CH";
var dataNode = "DT1.1";

In [ ]:
#!eval-notebook "../Initialization/InitSystemorphToMemory"

In [ ]:
Workspace.Initialize(x => x.FromSource(DataSource));

In [ ]:
var targetPartition = (await DataSource.Query<PartitionByReportingNodeAndPeriod>().Where(p => p.ReportingNode == reportingNode &&
                                                                                                   p.Year == year &&
                                                                                                   p.Month == month &&
                                                                                                   p.Scenario == scenario).ToArrayAsync()).Single().Id;
var defaultPartition = (await DataSource.Query<PartitionByReportingNodeAndPeriod>().Where(p => p.ReportingNode == reportingNode &&
                                                                                                    p.Year == year &&
                                                                                                    p.Month == month &&
                                                                                                    p.Scenario == null).ToArrayAsync()).Single().Id;


# Import Scenario Data

In [ ]:
#!import "../../ifrs17/Utils/ImportCalculationMethods"
#!import "../../ifrs17/Utils/Queries"

In [ ]:
public class ImportStorage
{   
    private readonly IDataSource querySource; 
    private readonly IWorkspace workspace;
    private readonly Systemorph.Vertex.Hierarchies.IHierarchicalDimensionCache hierarchyCache;
    private readonly ImportArgs args;
    
    // Constants
    private static int periodicityInMonths => 3; // Revisit
    
    //Format
    public string ImportFormat => args.ImportFormat; 
    
    //Time Periods 
    public (int Year, int Month) CurrentReportingPeriod => (args.Year, args.Month);
    public (int Year, int Month) PreviousReportingPeriod => (args.Year - 1, MonthInAYear); // YTD Logic

    //Partitions
    public Guid PartitionByRn;
    public Guid TargetPartition;
    public Guid DefaultPartition;
    public Guid PreviousPeriodPartition;    

    //Projections
    private ProjectionConfiguration[] ProjectionConfiguration;
    
    //DataNodes
    public IDictionary<string, DataNodeData> DataNodeDataBySystemName { get; private set; }
    public IDictionary<ImportScope, HashSet<string>> DataNodesByImportScope { get; private set; }
    public IDictionary<string, ICollection<int?>> AccidentYearsByDataNode { get; private set; }
    //Variables
    public IDictionary<string, ICollection<RawVariable>> RawVariablesByImportIdentity { get; private set; }
    public IDictionary<string, ICollection<IfrsVariable>> IfrsVariablesByImportIdentity { get; private set; }
    public RawVariable[] DefaultRawVariables { get; private set; }
    public HashSet<string> PrimaryScopeFromLinkedReinsurance { get; private set; }
        
    //Parameters
    public Dictionary<string, YieldCurve> LockedInYieldCurve { get; private set; }
    public Dictionary<string, Dictionary<int, YieldCurve>> CurrentYieldCurve { get; private set; }
    public Dictionary<string, Dictionary<int, PartnerRating>> PartnerRating { get; private set; }
    public Dictionary<string, Dictionary<int, CreditDefaultRate>> CreditDefaultRates { get; private set; }
    public Dictionary<string, Dictionary<int, SingleDataNodeParameter>> SingleDataNodeParametersByGoc { get; private set; }
    public Dictionary<string, Dictionary<int, HashSet<InterDataNodeParameter>>> InterDataNodeParametersByGoc { get; private set; }
    public Dictionary<AocStep, AocConfiguration> AocConfigurationByAocStep { get; private set; }
    
    private Dictionary<InputSource, HashSet<AocStep>> aocStepByInputSource;
    
    //Dimensions
    public Dictionary<string, AmountType> AmountTypeDimension { get; private set; }
    public Dictionary<string, Novelty> NoveltyDimension { get; private set; }
    public Dictionary<string, EstimateType> EstimateTypeDimension { get; private set; }
    public Dictionary<string, HashSet<string>> EstimateTypesByImportFormat { get; private set; }
    public Dictionary<string, string> ExperienceAdjustEstimateTypeMapping { get; private set; } 
       
    //Constructor
    public ImportStorage(ImportArgs args, IDataSource querySource, IWorkspace workspace)
    {
        this.querySource = querySource;
        this.workspace = workspace;
        hierarchyCache = workspace.ToHierarchicalDimensionCache();
        this.args = args;
    }
    
    //Initialize
    public async Task InitializeAsync()
    {   
        //Dimensions
        var estimateTypes = (await workspace.Query<EstimateType>().ToArrayAsync());
        
        EstimateTypeDimension     = estimateTypes.ToDictionary(x => x.SystemName);
        AmountTypeDimension       = (await workspace.Query<AmountType>().ToArrayAsync()).ToDictionary(x => x.SystemName);
        NoveltyDimension          = (await workspace.Query<Novelty>().ToArrayAsync()).ToDictionary(x => x.SystemName);
        
        ExperienceAdjustEstimateTypeMapping = new Dictionary<string,string>{{EstimateTypes.A, EstimateTypes.APA}, {EstimateTypes.AA, EstimateTypes.AAPA}, {EstimateTypes.OA, EstimateTypes.OAPA}}; //TODO move this logic
        
        //Hierarchy Cache
        hierarchyCache.Initialize<AmountType>();
        
        //EstimateType to load and to update
        EstimateTypesByImportFormat = new InputSource[] { InputSource.Opening, InputSource.Actual, InputSource.Cashflow,}
                                        .ToDictionary(x => x.ToString(), 
                                                      x => estimateTypes
                                                      .Where(et => et.InputSource.Contains(x))
                                                      .Select(et => et.SystemName)
                                                      .ToHashSet());
        
        //ProjectionConfiguration : Current Period + projection for every Quarter End for current Year and next Years as in projectionConfiguration.csv
        ProjectionConfiguration = (await workspace.Query<ProjectionConfiguration>()
                                               .Where(x => x.Shift > 0 || x.TimeStep == args.Month || (x.TimeStep > args.Month && x.TimeStep % periodicityInMonths == 0))
                                               .OrderBy(x => x.Shift).ThenBy(x => x.TimeStep).ToArrayAsync());
        
        //Get Partitions
        PartitionByRn = (await querySource.Query<PartitionByReportingNode>().Where(p => p.ReportingNode == args.ReportingNode).ToArrayAsync()).Single().Id;
        TargetPartition = (await querySource.Query<PartitionByReportingNodeAndPeriod>().Where(p => p.ReportingNode == args.ReportingNode &&
                                                                                                   p.Year == CurrentReportingPeriod.Year &&
                                                                                                   p.Month == CurrentReportingPeriod.Month &&
                                                                                                   p.Scenario == args.Scenario).ToArrayAsync()).Single().Id;
        DefaultPartition = (await querySource.Query<PartitionByReportingNodeAndPeriod>().Where(p => p.ReportingNode == args.ReportingNode &&
                                                                                                    p.Year == CurrentReportingPeriod.Year &&
                                                                                                    p.Month == CurrentReportingPeriod.Month &&
                                                                                                    p.Scenario == null).ToArrayAsync()).Single().Id;
        //Set Partitions
        await querySource.Partition.SetAsync<PartitionByReportingNode>(PartitionByRn);
        await workspace.Partition.SetAsync<PartitionByReportingNode>(PartitionByRn);
        
        await querySource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(TargetPartition);
        await workspace.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(TargetPartition);
        
        //Get data from Workspace (result of parsing)
        var parsedRawVariables = await workspace.Query<RawVariable>().ToArrayAsync();
        var parsedIfrsVariables = await workspace.Query<IfrsVariable>().ToArrayAsync();
        
        //DataNodes
        DataNodeDataBySystemName = await workspace.LoadDataNodesAsync(args);
        
        //Accident Years
        AccidentYearsByDataNode = (IDictionary<string, ICollection<int?>>)
            (ImportFormat == ImportFormats.Cashflow ? parsedRawVariables.Select(x => new {x.DataNode, x.AccidentYear}) : parsedIfrsVariables.Select(x => new {x.DataNode, x.AccidentYear}))
            .ToDictionaryGrouped(x => x.DataNode, x => (ICollection<int?>)x.Select(y => y.AccidentYear).ToHashSet());
        
        // Import Scopes and Data Node relationship parameters
        InterDataNodeParametersByGoc = await workspace.LoadInterDataNodeParametersAsync(args);
        
        var primaryScopeFromParsedVariables = (ImportFormat == ImportFormats.Cashflow ? parsedRawVariables.Select(x => x.DataNode) : parsedIfrsVariables.Select(x => x.DataNode)).ToHashSet();
        PrimaryScopeFromLinkedReinsurance = primaryScopeFromParsedVariables
                                            .Where(goc => !DataNodeDataBySystemName[goc].IsReinsurance && DataNodeDataBySystemName[goc].LiabilityType == LiabilityTypes.LRC)
                                            .SelectMany(goc => InterDataNodeParametersByGoc.TryGetValue(goc, out var interDataNodeParamByPeriod)
                                                                  ? interDataNodeParamByPeriod[CurrentPeriod].Select(param => param.DataNode == goc ? param.LinkedDataNode : param.DataNode).Where(goc => !primaryScopeFromParsedVariables.Contains(goc))
                                                                  : Enumerable.Empty<string>())
                                            .ToHashSet();
        
        var primaryScope = primaryScopeFromParsedVariables.Concat(PrimaryScopeFromLinkedReinsurance).ToHashSet();
        var secondaryScope = InterDataNodeParametersByGoc
                            .Where(kvp => primaryScope.Contains(kvp.Key))
                            .SelectMany(kvp => { var linkedGocs = kvp.Value[CurrentPeriod].Select(param => param.DataNode == kvp.Key ? param.LinkedDataNode : param.DataNode);
                                                return linkedGocs.Where(goc => !primaryScope.Contains(goc));}).ToHashSet();
        var allImportScopes = new HashSet<string>(primaryScope.Concat(secondaryScope));
        
        DataNodesByImportScope = new Dictionary<ImportScope, HashSet<string>> { { ImportScope.Primary, primaryScope }, { ImportScope.Secondary, secondaryScope } };
        
        // Parameters
        PartnerRating = await workspace.LoadCurrentAndPreviousParameterAsync<PartnerRating>(args, x => x.Partner);
        CreditDefaultRates = await workspace.LoadCurrentAndPreviousParameterAsync<CreditDefaultRate>(args, x => x.CreditRiskRating);
        SingleDataNodeParametersByGoc = await workspace.LoadSingleDataNodeParametersAsync(args);
        LockedInYieldCurve = await workspace.LoadLockedInYieldCurveAsync(args, allImportScopes.Select(dn => DataNodeDataBySystemName[dn]));
        CurrentYieldCurve = await workspace.LoadCurrentYieldCurveAsync(args, allImportScopes.Select(dn => DataNodeDataBySystemName[dn])); //TODO Rename this variable
        
        AocConfigurationByAocStep = await querySource.LoadAocStepConfigurationAsDictionaryAsync(args.Year, args.Month);         
        aocStepByInputSource = ((InputSource[])Enum.GetValues(typeof(InputSource))).ToDictionary(x => x,
                                                                                                 x => AocConfigurationByAocStep
                                                                                                 .Where(kvp => kvp.Value.InputSource.Contains(x))
                                                                                                 .Select(kvp => kvp.Key)
                                                                                                 .ToHashSet());
        
        //Previous Period
        var openingRawVariables = Enumerable.Empty<RawVariable>();
        var openingIfrsVariables = Enumerable.Empty<IfrsVariable>();

        if(allImportScopes.Select(dn => DataNodeDataBySystemName[dn]).Any(dnd => dnd.Year < args.Year)) {
            PreviousPeriodPartition = (await querySource.Query<PartitionByReportingNodeAndPeriod>().Where(p => p.ReportingNode == args.ReportingNode &&
                                                                                                           p.Year == PreviousReportingPeriod.Year &&
                                                                                                           p.Month == PreviousReportingPeriod.Month &&
                                                                                                           p.Scenario == null).ToArrayAsync()).Single().Id; // This can send an exception
            
            await querySource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(PreviousPeriodPartition);
        
            //Perform queries to previous Period
            openingRawVariables = (await querySource.Query<RawVariable>()
                                   .Where(rv => rv.Partition == PreviousPeriodPartition && rv.AocType == AocTypes.CL)
                                   .Where(v => primaryScope.Contains(v.DataNode))
                                   .ToArrayAsync())
                .Select(rv => rv with {AocType = AocTypes.BOP, Novelty = Novelties.I, Values = rv.Values.Skip(MonthInAYear).ToArray(), Partition = TargetPartition});
            
            openingIfrsVariables = (await querySource.Query<IfrsVariable>()
                                    .Where(iv => iv.Partition == PreviousPeriodPartition && iv.AocType == AocTypes.EOP)
                                    .Where(v => allImportScopes.Contains(v.DataNode))
                                    .ToArrayAsync())
                .Select(iv => iv with {AocType = AocTypes.BOP, Novelty = Novelties.I, Partition = TargetPartition});
            
            // TODO: print error if 
            //openingRawVariables.Select(x => x.DataNode).ToHashSet() != dataNodesWithPreviousPeriod
        }
        
        //SetPartition to current Period
        await querySource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(TargetPartition);
        
        if(primaryScope.Select(dn => DataNodeDataBySystemName[dn]).Any(dnd => dnd.Year == args.Year) && ImportFormat != ImportFormats.Opening)
            openingIfrsVariables = openingIfrsVariables.Concat(await querySource.Query<IfrsVariable>()
                                                                .Where(iv => iv.Partition == TargetPartition && iv.AocType == AocTypes.BOP && iv.Novelty == Novelties.I)
                                                                .Where(iv => primaryScope.Contains(iv.DataNode) && EstimateTypesByImportFormat[InputSource.Opening.ToString()].Contains(iv.EstimateType) || secondaryScope.Contains(iv.DataNode))
                                                                .ToArrayAsync());
        
        //Variables
        var rawVariables = parsedRawVariables.Concat(openingRawVariables)
                                             .Concat(await querySource.Query<RawVariable>()
                                                         .Where(rv => rv.Partition == TargetPartition)
                                                         .Where(rv => PrimaryScopeFromLinkedReinsurance.Contains(rv.DataNode))
                                                       .ToArrayAsync());

        if(DefaultPartition != TargetPartition){
            var orderedPartition = new List<Guid> {TargetPartition, DefaultPartition};
            await querySource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(DefaultPartition);
            var defaultRawVariables = await querySource.Query<RawVariable>()
                                                         .Where(rv => rv.Partition == DefaultPartition)
                                                         .Where(rv => primaryScopeFromParsedVariables.Contains(rv.DataNode)).ToArrayAsync();
            rawVariables = rawVariables.Concat(defaultRawVariables)
                                        .GroupBy(rv => 
                                             new {rv.DataNode, rv.AocType, rv.Novelty, rv.EstimateType, rv.AccidentYear, rv.Partition})
                                            .OrderBy(grv => orderedPartition.IndexOf(grv.Key.Partition))
                                            .Select(grv => grv.FirstOrDefault());
            DefaultRawVariables = defaultRawVariables;                                           
        }
        else {
            DefaultRawVariables = Enumerable.Empty<RawVariable>().ToArray();
        }
        
        RawVariablesByImportIdentity = (IDictionary<string, ICollection<RawVariable>>)rawVariables.ToDictionaryGrouped(v => v.DataNode, v => (ICollection<RawVariable>)v.ToArray());
        
        var ifrsVariables = parsedIfrsVariables.Concat(openingIfrsVariables)
                                                .Concat(await querySource.Query<IfrsVariable>()
                                                         .Where(iv => iv.Partition == TargetPartition && !(iv.AocType == AocTypes.BOP && iv.Novelty == Novelties.I)) //Actuals or Gross
                                                         .Where(iv => primaryScopeFromParsedVariables.Contains(iv.DataNode) && !EstimateTypesByImportFormat[ImportFormat].Contains(iv.EstimateType)  
                                                                || PrimaryScopeFromLinkedReinsurance.Contains(iv.DataNode) 
                                                                || secondaryScope.Contains(iv.DataNode))
                                                       .ToArrayAsync());
        IfrsVariablesByImportIdentity = (IDictionary<string, ICollection<IfrsVariable>>)ifrsVariables.ToDictionaryGrouped(v => v.DataNode, v => (ICollection<IfrsVariable>)v.ToArray());
    }
    
    //Getters
    
    //Periods
    public ValuationPeriod GetValuationPeriod(ImportIdentity id) => AocConfigurationByAocStep[new AocStep(id.AocType, id.Novelty)].ValuationPeriod;
    public PeriodType GetYieldCurvePeriod(ImportIdentity id) => AocConfigurationByAocStep[new AocStep(id.AocType, id.Novelty)].YcPeriod;
    public PeriodType GetCreditDefaultRiskPeriod(ImportIdentity id) => AocConfigurationByAocStep[new AocStep(id.AocType, id.Novelty)].CdrPeriod;
    
    public IEnumerable<AocStep> GetAllAocSteps(InputSource source) => aocStepByInputSource[source];
    public IEnumerable<AocStep> GetCalculatedTelescopicAocSteps() => AocConfigurationByAocStep.Where(kvp => kvp.Value.DataType == DataType.CalculatedTelescopic).Select(kvp => kvp.Key);
        
    //YieldCurve
    public double[] GetYearlyYieldCurve(ImportIdentity id, string economicBasis) {
        var yc = GetYieldCurve(id, economicBasis);
        return yc.Values.Skip(args.Year - yc.Year).ToArray(); //Check if the returned array is empty? Log Warning?
    }
    
    public YieldCurve GetYieldCurve(ImportIdentity id, string economicBasis) => (economicBasis, GetYieldCurvePeriod(id)) switch {
            (EconomicBases.C, PeriodType.BeginningOfPeriod) => CurrentYieldCurve[DataNodeDataBySystemName[id.DataNode].ContractualCurrency][PreviousPeriod],
            (EconomicBases.C, PeriodType.EndOfPeriod) => CurrentYieldCurve[DataNodeDataBySystemName[id.DataNode].ContractualCurrency][CurrentPeriod],
            (EconomicBases.L, _ ) => LockedInYieldCurve[id.DataNode],
            (_, PeriodType.NotApplicable) => (YieldCurve)ApplicationMessage.Log(Error.YieldCurvePeriodNotApplicable, id.AocType, id.Novelty),
            (_, _) => (YieldCurve)ApplicationMessage.Log(Error.EconomicBasisNotFound, id.DataNode)
       };
    
    //int Identity.ProjectionPeriod 
    public int GetProjectionCount() => ProjectionConfiguration.Count();
    public int GetShift(int projectionPeriod) => ProjectionConfiguration[projectionPeriod].Shift;
    public int GetTimeStep(int projectionPeriod) => ProjectionConfiguration[projectionPeriod].TimeStep;
    
    public PeriodType GetPeriodType(string amountType, string estimateType) => 
                    amountType != null && AmountTypeDimension.TryGetValue(amountType, out var at) 
                                ? at.PeriodType
                                : estimateType != null && EstimateTypeDimension.TryGetValue(estimateType, out var ct) 
                                    ? ct.PeriodType : PeriodType.EndOfPeriod;

    //Variables and Cashflows
    
    public IDictionary<string, ICollection<RawVariable>> TestRawVars() => RawVariablesByImportIdentity;
    public IDictionary<string, ICollection<IfrsVariable>> TestIfrsVars() => IfrsVariablesByImportIdentity;
    public IEnumerable<RawVariable> GetRawVariables(string dataNode) => RawVariablesByImportIdentity.TryGetValue(dataNode, out var variableCollection) ? variableCollection : Enumerable.Empty<RawVariable>();
    public IEnumerable<IfrsVariable> GetIfrsVariables(string dataNode) => IfrsVariablesByImportIdentity.TryGetValue(dataNode, out var variableCollection) ? variableCollection : Enumerable.Empty<IfrsVariable>();
    
    public double[] GetValues(ImportIdentity id, Func<RawVariable, bool> whereClause) => GetRawVariables(id.DataNode).Where(v => (v.AocType, v.Novelty) == id.AocStep && whereClause(v)).Aggregate()?.Values ?? Enumerable.Empty<double>().ToArray();
    public double GetValue(ImportIdentity id, Func<IfrsVariable, bool> whereClause) => GetIfrsVariables(id.DataNode).Where(v => (v.AocType, v.Novelty) == id.AocStep && whereClause(v)).Aggregate()?.Value ?? 0;
    
    public double[] GetValues(ImportIdentity id, string amountType, string estimateType, int? accidentYear) => GetValues(id, v => v.AccidentYear == accidentYear && v.AmountType == amountType && v.EstimateType == estimateType);
    public double GetValue(ImportIdentity id, string amountType, string estimateType, int? accidentYear) => GetValue(id, v => v.AccidentYear == accidentYear && v.AmountType == amountType && v.EstimateType == estimateType);
    public double GetValue(ImportIdentity id, string amountType, string estimateType, string economicBasis, int? accidentYear) => GetValue(id, v => v.AccidentYear == accidentYear && v.AmountType == amountType && v.EstimateType == estimateType && v.EconomicBasis == economicBasis);
   
    //Novelty
    private IEnumerable<string> GetNoveltiesForAocType(string aocType, IEnumerable<AocStep> aocConfiguration) => aocConfiguration.Where(aocStep => aocStep.AocType == aocType).Select(aocStep => aocStep.Novelty);
    public IEnumerable<string> GetNovelties() => NoveltyDimension.Keys;
    public IEnumerable<string> GetNovelties(string aocType) => GetNoveltiesForAocType(aocType, AocConfigurationByAocStep.Keys);
    public IEnumerable<string> GetNovelties(string aocType, InputSource inputSource) => GetNoveltiesForAocType(aocType, aocStepByInputSource[inputSource]);
    
    //Accident years
    public IEnumerable<int?> GetAccidentYears(string dataNode) => AccidentYearsByDataNode.TryGetValue(dataNode, out var accidentYear) ? accidentYear : new int?[] { null };
    
    // Parameters
    public double GetNonPerformanceRiskRate (ImportIdentity identity) {
        var period = GetCreditDefaultRiskPeriod(identity) == PeriodType.BeginningOfPeriod ? PreviousPeriod : CurrentPeriod;
       
        if(!DataNodeDataBySystemName.TryGetValue(identity.DataNode, out var dataNodeData))     ApplicationMessage.Log(Error.DataNodeNotFound, identity.DataNode);
        if(dataNodeData.Partner == null)                                                       ApplicationMessage.Log(Error.PartnerNotFound, identity.DataNode);
        // if Partner == Internal then return 0;
        if(!PartnerRating.TryGetValue(dataNodeData.Partner, out var rating))                   ApplicationMessage.Log(Error.RatingNotFound, dataNodeData.Partner);
        if(!CreditDefaultRates.TryGetValue(rating[period].CreditRiskRating, out var rate))     ApplicationMessage.Log(Error.CreditDefaultRateNotFound, rating[period].CreditRiskRating);
        return rate[period].Values[0];
    }
    
    public double GetPremiumAllocationFactor(ImportIdentity id) => SingleDataNodeParametersByGoc.TryGetValue(id.DataNode, out var singleDataNodeParameter) 
                                                                                ? singleDataNodeParameter[CurrentPeriod].PremiumAllocation : DefaultPremiumExperienceAdjustmentFactor;
    
    // Data Node relationships
    public IEnumerable<string> GetUnderlyingGic(ImportIdentity id) => !InterDataNodeParametersByGoc.TryGetValue(id.DataNode, out var interDataNodeParameters)
        ? Enumerable.Empty<string>()
        : interDataNodeParameters[CurrentPeriod].Select(x => x.DataNode != id.DataNode ? x.DataNode : x.LinkedDataNode).Where(goc => !DataNodeDataBySystemName[goc].IsReinsurance);
    
    public double GetReinsuranceCoverage (ImportIdentity id, string gic)  
    {
        var targetPeriod = AocConfigurationByAocStep[new AocStep(id.AocType, id.Novelty)].RcPeriod == PeriodType.EndOfPeriod ? CurrentPeriod : PreviousPeriod;
        return InterDataNodeParametersByGoc.TryGetValue(id.DataNode, out var interDataNodeParameters)
            ? interDataNodeParameters[targetPeriod].FirstOrDefault(x => x.DataNode == gic || x.LinkedDataNode == gic).ReinsuranceCoverage
            : (double)ApplicationMessage.Log(Error.ReinsuranceCoverage, id.DataNode);
    }
    
    // Import Scope
    public bool IsPrimaryScope (string dataNode) => DataNodesByImportScope[ImportScope.Primary].Contains(dataNode);
    public bool IsSecondaryScope (string dataNode) => DataNodesByImportScope[ImportScope.Secondary].Contains(dataNode);
    
    // Other
    public IEnumerable<string> GetNonAttributableAmountType() => new string[]{AmountTypes.NE};
    public IEnumerable<string> GetAttributableExpenseAndCommissionAmountType() => new string[] {AmountTypes.ACA, AmountTypes.AEA}; //U+ specific
    public IEnumerable<string> GetInvestmentClaims() => hierarchyCache.Get<AmountType>(AmountTypes.ICO).Descendants(includeSelf : true).Select(x => x.SystemName);
    public IEnumerable<string> GetPremiums() => hierarchyCache.Get<AmountType>(AmountTypes.PR).Descendants(includeSelf : true).Select(x => x.SystemName);
    public IEnumerable<string> GetClaims() => hierarchyCache.Get<AmountType>(AmountTypes.CL).Descendants().Select(x => x.SystemName);
}

In [ ]:
var fileName = "../Files/900.TransactionalData/NominalCashflows_CH_2020_12_Scenario.csv";
Activity.Start();
var args = (await GetArgsFromMainAsync<PartitionByReportingNodeAndPeriod>(fileName)) with {ImportFormat = ImportFormats.Cashflow};
await DataNodeFactoryAsync(fileName, ImportFormats.Cashflow, args);
if(Activity.HasErrors()) return Activity.Finish();
    
var parsingLog = await ParseCashflowsToWorkspaceAsync(fileName, args);
if(parsingLog.Errors.Any()) return Activity.Finish().Merge(parsingLog);
    
var storage = new ImportStorage(args, DataSource, Workspace);
await storage.InitializeAsync();
var test = storage.TestRawVars();

await DataSource.Partition.SetAsync<PartitionByReportingNodeAndPeriod>(targetPartition);
var defaultRawVariables = await DataSource.Query<RawVariable>()
                                                         .Where(rv => rv.Partition == targetPartition)
                                                         .ToArrayAsync();
test[dataNode].GroupBy(x => x.Partition).Select(x => x)
/*
.ToDictionaryGrouped( x => x.Partition, x=> x).Keys;
rawDictScenario
                    /*.GroupBy(rv => ImportIdentity)
                    .Select(rv => rv.OrderByDescending(x => x.Scenario)
                                    .FirstOrDefault()).ToArray();
if(Activity.HasErrors()) return Activity.Finish().Merge(parsingLog);
var idGroups = rawDictScenario[dataNode].ToDictionaryGrouped( x => x.Partition, x=> x);
var universe = Scopes.ForStorage(storage).ToScope<IModel>();
var identities = universe.GetScopes<GetIdentities>(storage.DataNodesByImportScope[ImportScope.Primary]).SelectMany(s => s.Identities);
var ivs = universe.GetScopes<ComputeAllScopes>(identities).SelectMany(x => x.CalculatedIfrsVariables);
if(Activity.HasErrors()) return Activity.Finish().Merge(parsingLog);
var test = ivs.ToDictionaryGrouped( x => x.Partition, x=> x);
test
await Workspace.UpdateAsync<IfrsVariable>(ivs);
await CommitToDatabase<IfrsVariable>(storage.TargetPartition, 
                                         snapshot : true, 
                                         filter : x => storage.EstimateTypesByImportFormat[ImportFormats.Cashflow].Contains(x.EstimateType) && 
                                                       storage.DataNodesByImportScope[ImportScope.Primary].Contains(x.DataNode));  
await CommitToDatabase<RawVariable>(storage.TargetPartition, 
                                        snapshot : true,
                                        filter : x => storage.DataNodesByImportScope[ImportScope.Primary].Contains(x.DataNode));


var idGroups = ifrsDict[dataNode].ToDictionaryGrouped( x => x.Id, x=> x);
idGroups.Keys
var test = ifrsDict[dataNode].ToList().Select(x => new { x.Id == defif(x.Id == targetPartition || x.Id == defaultPartition) new x : null});*/

In [ ]:
var fileName = "../Files/900.TransactionalData/NominalCashflows_CH_2020_12.csv";
Activity.Start();
var args = (await GetArgsFromMainAsync<PartitionByReportingNodeAndPeriod>(fileName)) with {ImportFormat = ImportFormats.Cashflow};
await DataNodeFactoryAsync(fileName, ImportFormats.Cashflow, args);
if(Activity.HasErrors()) return Activity.Finish();
    
var parsingLog = await ParseCashflowsToWorkspaceAsync(fileName, args);
if(parsingLog.Errors.Any()) return Activity.Finish().Merge(parsingLog);
    
var storage = new ImportStorage(args, DataSource, Workspace);
await storage.InitializeAsync();
var rawDictBE = storage.TestRawVars();
if(Activity.HasErrors()) return Activity.Finish().Merge(parsingLog);
var idGroups = rawDictBE[dataNode].ToDictionaryGrouped( x => x.Partition, x=> x);
idGroups.Keys